In [1]:
import torch
from torchvision import transforms
from PIL import Image
import torchvision.models as models
import torch.nn as nn

In [39]:
model = models.resnet50(pretrained=False)  # Assuming you've trained and saved the model
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 5),  # Assuming 5 output nodes for 5 defect types
    nn.Sigmoid()
)

c:\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [40]:
model.load_state_dict(torch.load('skin_defects_model.pth'))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [44]:
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Lambda(lambda x: x.convert('RGB') if x.mode != 'RGB' else x),  # Convert to RGB if not already
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0) == 1 else x),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [45]:
def predict_skin_defects(image_path):
    image = Image.open(image_path)
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        outputs = model(image_tensor)
    probabilities = outputs.squeeze().cpu().numpy()
    return probabilities


In [48]:
classes = ['Acne','Blackhead','Open Pores','Scars','Whitehead']
def findProblems(probabilities):
    ans = []
    for i in range(5):
        if probabilities[i] >= 0.1:
            ans.append(classes[i])
    if(len(ans)==0):
        ans.append('Normal Skin with no defect')
    return ans
        

In [56]:
image_path = 'check1.jpg'
probabilities = predict_skin_defects(image_path)

In [57]:
print('Predicted probabilities for each defect type:', findProblems(probabilities))

Predicted probabilities for each defect type: ['Acne', 'Scars']


In [70]:
image_path2 = 'manycheck2.jpg'
probabilities2 = predict_skin_defects(image_path2)

In [71]:
print('Predicted probabilities for each defect type:', findProblems(probabilities2))

Predicted probabilities for each defect type: ['Acne', 'Scars']
